In [1]:
# Carregar os módulos necessários para análise
import pandas as pd
import folium
from folium.plugins import HeatMap
from tqdm import tqdm
import numpy as np
from locale import atof

In [2]:
# Importando os dados
# Dados foram extraídos de um experimento a partir de:
# https://github.com/pires/android-obd-reader
df_vehicle = pd.read_csv("Log_23_08_2017_19_26_17.csv", 
                   encoding = 'latin2',
                   sep=';',
                   skiprows=[0],
                   low_memory=False)

In [5]:
# Visualizando as cinco primeiras linhas da base de dados
df_vehicle.head()

,TIME,LATITUDE,LONGITUDE,ALTITUDE,VEHICLE_ID,BAROMETRIC_PRESSURE,ENGINE_COOLANT_TEMP,FUEL_LEVEL,ENGINE_LOAD,AMBIENT_AIR_TEMP,...,FUEL_PRESSURE,SPEED,Short Term Fuel Trim Bank 2,Short Term Fuel Trim Bank 1,ENGINE_RUNTIME,THROTTLE_POS,DTC_NUMBER,TROUBLE_CODES,TIMING_ADVANCE,EQUIV_RATIO
0,1503527187655,0.0,0.0,0.0,goluefs,null,90C,null,42.0%,null,...,NODATA,47km/h,null,-9.4%,NODATA,22.0%,null,NaN,null,null
1,1503527189942,0.0,0.0,0.0,goluefs,null,90C,null,17.6%,null,...,NODATA,48km/h,null,19.5%,NODATA,16.9%,null,NaN,null,null
2,1503527192664,0.0,0.0,0.0,goluefs,null,90C,null,54.9%,null,...,NODATA,51km/h,null,-8.6%,NODATA,22.0%,null,NaN,null,null
3,1503527194963,0.0,0.0,0.0,goluefs,null,90C,null,47.8%,null,...,NODATA,55km/h,null,-21.1%,NODATA,18.8%,null,NaN,null,null
4,1503527197684,0.0,0.0,0.0,goluefs,null,90C,null,35.7%,null,...,NODATA,55km/h,null,0.0%,NODATA,20.0%,null,NaN,null,null


In [6]:
# Substituindo a string "null" e "canerror" na base de dados por NaN - not a number
df_vehicle.replace('null',np.NaN, inplace=True)
df_vehicle.replace('CANERROR',np.NaN, inplace=True)
df_vehicle.head()

,TIME,LATITUDE,LONGITUDE,ALTITUDE,VEHICLE_ID,BAROMETRIC_PRESSURE,ENGINE_COOLANT_TEMP,FUEL_LEVEL,ENGINE_LOAD,AMBIENT_AIR_TEMP,...,FUEL_PRESSURE,SPEED,Short Term Fuel Trim Bank 2,Short Term Fuel Trim Bank 1,ENGINE_RUNTIME,THROTTLE_POS,DTC_NUMBER,TROUBLE_CODES,TIMING_ADVANCE,EQUIV_RATIO
0,1503527187655,0.0,0.0,0.0,goluefs,NaN,90C,NaN,42.0%,NaN,...,NODATA,47km/h,NaN,-9.4%,NODATA,22.0%,NaN,NaN,NaN,NaN
1,1503527189942,0.0,0.0,0.0,goluefs,NaN,90C,NaN,17.6%,NaN,...,NODATA,48km/h,NaN,19.5%,NODATA,16.9%,NaN,NaN,NaN,NaN
2,1503527192664,0.0,0.0,0.0,goluefs,NaN,90C,NaN,54.9%,NaN,...,NODATA,51km/h,NaN,-8.6%,NODATA,22.0%,NaN,NaN,NaN,NaN
3,1503527194963,0.0,0.0,0.0,goluefs,NaN,90C,NaN,47.8%,NaN,...,NODATA,55km/h,NaN,-21.1%,NODATA,18.8%,NaN,NaN,NaN,NaN
4,1503527197684,0.0,0.0,0.0,goluefs,NaN,90C,NaN,35.7%,NaN,...,NODATA,55km/h,NaN,0.0%,NODATA,20.0%,NaN,NaN,NaN,NaN


In [7]:
# Verificando as colunas que fazem sentido explorar
df_vehicle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1347 entries, 0 to 1346
Data columns (total 28 columns):
TIME                           1347 non-null int64
LATITUDE                       1347 non-null float64
LONGITUDE                      1347 non-null float64
ALTITUDE                       1347 non-null float64
VEHICLE_ID                     1347 non-null object
BAROMETRIC_PRESSURE            0 non-null float64
ENGINE_COOLANT_TEMP            1347 non-null object
FUEL_LEVEL                     0 non-null float64
ENGINE_LOAD                    1347 non-null object
AMBIENT_AIR_TEMP               0 non-null float64
ENGINE_RPM                     1347 non-null object
INTAKE_MANIFOLD_PRESSURE       1347 non-null object
MAF                            0 non-null float64
Term Fuel Trim Bank 1          0 non-null float64
FUEL_ECONOMY                   0 non-null float64
Long Term Fuel Trim Bank 2     0 non-null float64
FUEL_TYPE                      0 non-null float64
AIR_INTAKE_TEMP        

In [8]:
df_vehicle.columns

Index(['TIME', 'LATITUDE', 'LONGITUDE', 'ALTITUDE', 'VEHICLE_ID',
       'BAROMETRIC_PRESSURE', 'ENGINE_COOLANT_TEMP', 'FUEL_LEVEL',
       'ENGINE_LOAD', 'AMBIENT_AIR_TEMP', 'ENGINE_RPM',
       'INTAKE_MANIFOLD_PRESSURE', 'MAF', 'Term Fuel Trim Bank 1',
       'FUEL_ECONOMY', 'Long Term Fuel Trim Bank 2', 'FUEL_TYPE',
       'AIR_INTAKE_TEMP', 'FUEL_PRESSURE', 'SPEED',
       'Short Term Fuel Trim Bank 2', 'Short Term Fuel Trim Bank 1',
       'ENGINE_RUNTIME', 'THROTTLE_POS', 'DTC_NUMBER', 'TROUBLE_CODES',
       'TIMING_ADVANCE', 'EQUIV_RATIO'],
      dtype='object')

In [13]:
# Limitando as colunas
df_vehicle = df_vehicle[[
    'TIME',
    'LATITUDE',
    'LONGITUDE',
    'ALTITUDE',
    'ENGINE_COOLANT_TEMP',
    'ENGINE_LOAD',
    'ENGINE_RPM',
    'SPEED',
    'THROTTLE_POS',
]
]


In [14]:
# Analisando a nova base de dados
df_vehicle.head()

,TIME,LATITUDE,LONGITUDE,ALTITUDE,ENGINE_COOLANT_TEMP,ENGINE_LOAD,ENGINE_RPM,SPEED,THROTTLE_POS
0,1503527187655,0.0,0.0,0.0,90C,42.0%,1539RPM,47km/h,22.0%
1,1503527189942,0.0,0.0,0.0,90C,17.6%,1702RPM,48km/h,16.9%
2,1503527192664,0.0,0.0,0.0,90C,54.9%,1729RPM,51km/h,22.0%
3,1503527194963,0.0,0.0,0.0,90C,47.8%,1901RPM,55km/h,18.8%
4,1503527197684,0.0,0.0,0.0,90C,35.7%,1934RPM,55km/h,20.0%


In [15]:
# A nova base de dados ainda contem dados faltantes
df_vehicle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1347 entries, 0 to 1346
Data columns (total 9 columns):
TIME                   1347 non-null int64
LATITUDE               1347 non-null float64
LONGITUDE              1347 non-null float64
ALTITUDE               1347 non-null float64
ENGINE_COOLANT_TEMP    1347 non-null object
ENGINE_LOAD            1347 non-null object
ENGINE_RPM             1347 non-null object
SPEED                  1339 non-null object
THROTTLE_POS           1347 non-null object
dtypes: float64(3), int64(1), object(5)
memory usage: 94.8+ KB


In [29]:
# Eliminar as linhas com algum valor nulo
df_vehicle.dropna(axis=0, inplace=True)
df_vehicle.reset_index(drop=True,inplace=True)

# Chegamos em uma base de dados higienizada
df_vehicle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1305 entries, 0 to 1304
Data columns (total 9 columns):
TIME                   1305 non-null int64
LATITUDE               1305 non-null float64
LONGITUDE              1305 non-null float64
ALTITUDE               1305 non-null float64
ENGINE_COOLANT_TEMP    1305 non-null int64
ENGINE_LOAD            1305 non-null float64
ENGINE_RPM             1305 non-null int64
SPEED                  1305 non-null int64
THROTTLE_POS           1305 non-null float64
dtypes: float64(5), int64(4)
memory usage: 91.8 KB


In [30]:
# Observe que alugmas colunas possuem valores com unidades
df_vehicle.head()

,TIME,LATITUDE,LONGITUDE,ALTITUDE,ENGINE_COOLANT_TEMP,ENGINE_LOAD,ENGINE_RPM,SPEED,THROTTLE_POS
0,1503527272916,-12.901591,-38.362720,12.883922,88,47.8,2224,80,22.7
1,1503527275145,-12.901591,-38.362720,12.883922,87,35.7,2239,80,19.6
2,1503527277929,-12.900811,-38.363287,9.855026,87,34.9,2234,80,20.8
3,1503527280157,-12.900811,-38.363287,9.855026,87,31.0,2196,78,18.4
4,1503527282947,-12.900811,-38.363287,9.855026,87,35.7,2181,78,20.0


In [19]:
df_vehicle = df_vehicle[df_vehicle['LATITUDE'] != 0.0]

In [20]:
df_vehicle.head()

,TIME,LATITUDE,LONGITUDE,ALTITUDE,ENGINE_COOLANT_TEMP,ENGINE_LOAD,ENGINE_RPM,SPEED,THROTTLE_POS
34,1503527272916,-12.901591,-38.362720,12.883922,88C,47.8%,2224RPM,80km/h,22.7%
35,1503527275145,-12.901591,-38.362720,12.883922,87C,35.7%,2239RPM,80km/h,19.6%
36,1503527277929,-12.900811,-38.363287,9.855026,87C,34.9%,2234RPM,80km/h,20.8%
37,1503527280157,-12.900811,-38.363287,9.855026,87C,31.0%,2196RPM,78km/h,18.4%
38,1503527282947,-12.900811,-38.363287,9.855026,87C,35.7%,2181RPM,78km/h,20.0%


In [21]:
# Eliminando caracteres indesejáveis (C, %, km/h, RPM)
df_vehicle['ENGINE_COOLANT_TEMP'] = df_vehicle['ENGINE_COOLANT_TEMP'].map(lambda x: x.rstrip('C'))
df_vehicle['ENGINE_LOAD'] = df_vehicle['ENGINE_LOAD'].map(lambda x: x.rstrip('%'))
df_vehicle['SPEED'] = df_vehicle['SPEED'].map(lambda x: x.rstrip('km/h'))
df_vehicle['THROTTLE_POS'] = df_vehicle['THROTTLE_POS'].map(lambda x: x.rstrip('%'))
df_vehicle['ENGINE_RPM'] = df_vehicle['ENGINE_RPM'].map(lambda x: x.rstrip('RPM'))
#df_vehicle['MAF'] = df_vehicle['MAF'].map(lambda x: x.rstrip('g/s'))


In [28]:
# Dado higienizados
df_vehicle.head()

,TIME,LATITUDE,LONGITUDE,ALTITUDE,ENGINE_COOLANT_TEMP,ENGINE_LOAD,ENGINE_RPM,SPEED,THROTTLE_POS
34,1503527272916,-12.901591,-38.362720,12.883922,88,47.8,2224,80,22.7
35,1503527275145,-12.901591,-38.362720,12.883922,87,35.7,2239,80,19.6
36,1503527277929,-12.900811,-38.363287,9.855026,87,34.9,2234,80,20.8
37,1503527280157,-12.900811,-38.363287,9.855026,87,31.0,2196,78,18.4
38,1503527282947,-12.900811,-38.363287,9.855026,87,35.7,2181,78,20.0


In [26]:
#cast
df_vehicle['ENGINE_COOLANT_TEMP'] = df_vehicle['ENGINE_COOLANT_TEMP'].astype(np.int64)
df_vehicle['ENGINE_LOAD'] = df_vehicle['ENGINE_LOAD'].astype(np.float64)
df_vehicle['ENGINE_RPM'] = df_vehicle['ENGINE_RPM'].astype(np.int64)
#df_vehicle['MAF'] = df_vehicle['MAF'].astype(np.float64)
df_vehicle['SPEED'] = df_vehicle['SPEED'].astype(np.int64)
df_vehicle['THROTTLE_POS'] = df_vehicle['THROTTLE_POS'].astype(np.float64)

In [161]:
# se for necessário salvar a base de dados
# df_vehicle.to_csv('dataset_higienizado.csv')

In [31]:
# Criar uma lista com [[lat,log,propriedade]]
coordinates = []
for i in tqdm(range(len(df_vehicle))):
        coordinates.append([df_vehicle.ix[i,'LATITUDE'], 
                        df_vehicle.ix[i,'LONGITUDE'], 
                        df_vehicle.ix[i,'SPEED']])
    

100%|██████████| 1305/1305 [00:00<00:00, 41066.02it/s]


In [35]:
#create map object
map_speed = folium.Map(
    location = [-12.901591, -38.362720],
    zoom_start= 12
)

# Valor máximo da propriedade com o intuito de normalizar os graficos
max_speed = df_vehicle['ENGINE_LOAD'].max()
print(max_speed)

HeatMap(coordinates,max_val=max_speed).add_to(map_speed)
map_speed

98.8
